This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [67]:
import pandas as pd
import numpy as np


file_name = "data/data_000637.txt"


data = pd.read_csv(file_name, nrows=11000)


data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
10995,1,0,139,3869200277,204,0
10996,1,0,61,3869200277,202,20
10997,1,0,63,3869200277,204,1
10998,1,0,59,3869200277,283,29


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [68]:
data['X'] = data['ORBIT_CNT'] / data['BX_COUNTER']
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,X
0,1,0,123,3869200167,2374,26,1.629823e+06
1,1,0,124,3869200167,2374,27,1.629823e+06
2,1,0,63,3869200167,2553,28,1.515550e+06
3,1,0,64,3869200167,2558,19,1.512588e+06
4,1,0,64,3869200167,2760,25,1.401884e+06
...,...,...,...,...,...,...,...
10995,1,0,139,3869200277,204,0,1.896667e+07
10996,1,0,61,3869200277,202,20,1.915446e+07
10997,1,0,63,3869200277,204,1,1.896667e+07
10998,1,0,59,3869200277,283,29,1.367209e+07


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [119]:
start = data.min(axis=0)
end = data.max(axis=0)
start_time = data['TDC_MEAS'][0] * 25 / 30 + data['BX_COUNTER'][0] * 25 + data['ORBIT_CNT'][0]*data['X'][0]*25
end_time = data['TDC_MEAS'][len(data)-1] * 25 / 30 + data['BX_COUNTER'][len(data)-1] * 25 + data['ORBIT_CNT'][len(data)-1]*data['X'][len(data)-1]*25

total_duration = (end_time - start_time) * pow(10,-9)
h = total_duration/3600
m = h%1/60
s = m%1
print("h:",h," m:",m, " s:", s)


KeyError: 'X'

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [70]:
data['TIMENS'] = data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25 + data['ORBIT_CNT']*data['X']*25
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,X,TIMENS
0,1,0,123,3869200167,2374,26,1.629823e+06,1.576528e+17
1,1,0,124,3869200167,2374,27,1.629823e+06,1.576528e+17
2,1,0,63,3869200167,2553,28,1.515550e+06,1.465992e+17
3,1,0,64,3869200167,2558,19,1.512588e+06,1.463126e+17
4,1,0,64,3869200167,2760,25,1.401884e+06,1.356043e+17
...,...,...,...,...,...,...,...,...
10995,1,0,139,3869200277,204,0,1.896667e+07,1.834646e+18
10996,1,0,61,3869200277,202,20,1.915446e+07,1.852811e+18
10997,1,0,63,3869200277,204,1,1.896667e+07,1.834646e+18
10998,1,0,59,3869200277,283,29,1.367209e+07,1.322501e+18


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [112]:
channels = data.groupby(['TDC_CHANNEL']).sum(['ORBIT_CNT','BX_COUNTER'])
channels['TOTAL'] = channels['ORBIT_CNT'] + channels['BX_COUNTER']
channels = channels.sort_values(by=['TOTAL'], ascending=False)
print(channels[:3], '\n')

             HEAD  FPGA      ORBIT_CNT  BX_COUNTER  TDC_MEAS          TOTAL
TDC_CHANNEL                                                                
139           941   288  3640917408275     1665503         0  3640919073778
64            554    13  2143536924351     1034779      7999  2143537959130
63            548     3  2120321722209     1006842      8241  2120322729051 



6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [117]:
channels_not_empty = 0
for i in range(len(data)):
    row = data.loc[data.index[i]]
    print(row['ORBIT_CNT'])

3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200167
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168
3869200168

3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200181
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182
3869200182

3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200193
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194
3869200194

3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200204
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205
3869200205

3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200216
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217
3869200217

3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200228
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229
3869200229

3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200237
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238
3869200238

3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200249
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250
3869200250

3869200261
3869200261
3869200261
3869200261
3869200261
3869200261
3869200261
3869200261
3869200261
3869200261
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262
3869200262

3869200273
3869200273
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200274
3869200275
3869200275
3869200275
3869200275
3869200275

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel